In [0]:
from pyspark import SparkConf, SparkContext

In [0]:
#there is an existing SparkContext object when we start Databricks notebook
print(sc)

<SparkContext master=local[8] appName=Databricks Shell>


<b>
Reading a file which gives temperatures per weather station from year 1800

In [0]:
lines = sc.textFile("dbfs:/FileStore/1800.csv")

<b>Defining a function to parse each line from the file</b>

In [0]:
def parseLines(line):
  fields = line.split(',')#field will be list of list
  print(type(fields))
  #extract individual fields
  stationId = fields[0]
  weatherDate = fields[1]
  measure = fields[2]
  temp = float(float(fields[3]) * 0.1) # original value is in tenth of degree Celsius
  return stationId, weatherDate, measure,temp

parsedLines  = lines.map(parseLines) #as we need to call #the above function for each item in the lines list we #use map
  

<b>
Defining a function to printRDD<br>
You need first collect() all elements into Driver and then loop through it to print

In [0]:
def printRDDtop5(rddval):
  rddvalues = rddval.collect()
  i = 0
  for _ in rddvalues:
    if i <= 5:
      print(_)
      i = i+1
      
def printRDDfull(rddval):
  rddvalues = rddval.collect()
  for _ in rddvalues:
    print(_)

In [0]:
printRDDtop5(parsedLines)

('ITE00100554', '18000101', 'TMAX', -7.5)
('ITE00100554', '18000101', 'TMIN', -14.8)
('GM000010962', '18000101', 'PRCP', 0.0)
('EZE00100082', '18000101', 'TMAX', -8.6)
('EZE00100082', '18000101', 'TMIN', -13.5)
('ITE00100554', '18000102', 'TMAX', -6.0)


<b> Now lets try to filter the RDD to get only the Minimum temperature for each station

In [0]:
minTemps = parsedLines.filter(lambda x : 'TMIN' in x[2])

In [0]:
printRDDtop5(minTemps)

('ITE00100554', '18000101', 'TMIN', -14.8)
('EZE00100082', '18000101', 'TMIN', -13.5)
('ITE00100554', '18000102', 'TMIN', -12.5)
('EZE00100082', '18000102', 'TMIN', -13.0)
('ITE00100554', '18000103', 'TMIN', -4.6000000000000005)
('EZE00100082', '18000103', 'TMIN', -7.300000000000001)


<b>Now lets create another RDD with only stationID and Temperatures only

In [0]:
minTemps = minTemps.map(lambda x: (x[0],x[3]))

In [0]:
#this essentially goes into loop, for each pair of entries x and y it tries to find the minimum
#finally only the minimum of the temperature survives for each key which in this case is the first column by default(statioID)
minTemps = minTemps.reduceByKey(lambda x,y : min(x,y) )

In [0]:
printRDDfull(minTemps)

('ITE00100554', -14.8)
('EZE00100082', -13.5)
